# 02. 텍스트 데이터 다루기

## 2.2 텍스트 토큰화하기

In [4]:
! uv pip install tiktoken

Using Python 3.12.11 environment at: /Users/hhkim/Study/LLMs-from-scratch-hands-on/.venv
Audited 1 package in 46ms


In [5]:
import urllib.request
url = ("https://raw.githubusercontent.com/rickiepark/"
        "llm-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x1049f5cd0>)

In [6]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("총 문자 개수:", len(raw_text))
print(raw_text[:99])

총 문자 개수: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [7]:
import re  # 파이썬 정규표현식 라이브러리
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text )  # 공백을 기준으로 텍스트 분할
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [8]:
result = re.split(r'([,.]|\s)', text )  # , . 또는 공백을 기준으로 텍스트 분할
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [9]:
result = [item for item in result if item.strip()]  # 공백 삭제
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [10]:
# 다양한 특수 문자 토큰화 테스트
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)  # text 토큰화
result = [item.strip() for item in result if item.strip()]  # 공백 삭제
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [11]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)  # the-verdict 소설 토큰화
preprocessed = [item.strip() for item in preprocessed if item.strip()]  # 공백 삭제
print(len(preprocessed))  # 공백을 제외한 텍스트에 있는 토큰의 개수

4690


In [12]:
print(preprocessed[:30])  # 처음 30개 토큰 출력

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## 2.3 토큰을 토큰 ID로 변환하기

In [ ]:
# 어휘 사전 구축
all_words = sorted(set(preprocessed))  # set으로 중복 제거 후 알파벳 순 정렬
vocab_size = len(all_words)  # 어휘 사전의 개수
print(vocab_size)

1130


In [14]:
vocab = {token:integer for integer, token in enumerate(all_words)}  # 어휘 사전에서 token을 int(ID)와 짝지어 딕셔너리 저장
for i, item in enumerate(vocab.items()):  # 딕셔너리에서 순서대로 51개 출력
	print(item)
	if i >= 50:
		break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [ ]:
# 어휘 사전을 새로운 텍스트에 적용하여 토큰 ID로 변환
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab  # encode 메서드와 decode 메서드에서 참조할 수 있도록 어휘사전을 클래스의 속성으로 저장
        self.int_to_str = {i:s for s,i in vocab.items()}  # 토큰 ID를 원본 텍스트 토큰으로 매핑하는 역어휘사전 생성

    # 입력 텍스트를 처리하여 토큰 ID로 변경
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)  # 입력받은 text를 토큰화
        preprocessed = [  # 공백 삭제
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]  # 토큰화 된 텍스트에서 한 단어씩 어휘사전에서 검색후 id 반환
        return ids
    
    # 토큰 ID를 텍스트로 되돌리기
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])  # text에 ids에서 정수를 하나씩 역어휘사전에서 검색 후 str로 변환해서 추가함(공백으로 구분)
        # return text
        # 지정된 구두점 문자 앞의 공백 삭제
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)  # \s과 캡처그룹()이 함께 있으면 캡처그룹만 남겨서 공백을 삭제하는 정규식
        return text

In [39]:
tokenizer = SimpleTokenizerV1(vocab)  # 토크나이저 객체 생성
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)  # 토큰화
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [40]:
print(tokenizer.decode(ids))  # 토큰 ID를 다시 텍스트로 변경

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [ ]:
text = "Hello, do you like tea?"  # 어휘사전에 없는 단어(Hello) 검색
print(tokenizer.encode(text))

KeyError: 'Hello'

## 2.4 특수 문맥 토큰 추가하기

In [ ]:
all_tokens = sorted(list(set(preprocessed)))  # 토큰화한 verdict 소설을 set으로 중복 제거 후 list로 만들어 정렬
all_tokens.extend(["<|endoftext|>", "<|unk|>"])  # 토큰 추가
vocab = {token:integer for integer,token in enumerate(all_tokens)}  # 어휘사전 생성

print(len(vocab.items()))  # 토큰 2개 추가된 1132개

1132


In [ ]:
for i,item in enumerate(list(vocab.items())[-5:]):  # 어휘사전에서 마지막 5개 출력
    print(item)  # endoftext, unk 토큰 확인

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [ ]:
# 모르는 단어를 처리하는 토크나이저
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab  # 어휘사전
        self.int_to_str = { i:s for s,i in vocab.items() }  # 역어휘사전

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)  # text 토큰화
        preprocessed = [ item.strip() for item in preprocessed if item.strip() ]  # 공백 제거
        preprocessed = [ item if item in self.str_to_int else "<|unk|>" for item in preprocessed ]  # 모르는 단어를 unk 토큰으로 변환

        ids = [ self.str_to_int[s] for s in preprocessed ]  # 단어 토큰을 ID로 변환
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])  # ID를 토큰으로 변환 (공백으로 구분)
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)  # 공백+구두점으로 되어있으면 공백 삭제
        return text

In [ ]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))  # endoftext 토큰으로 구분해서 text 합치기
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [ ]:
tokenizer = SimpleTokenizerV2(vocab)  # 어휘사전 객체 생성
print(tokenizer.encode(text))  # 인코딩 (1131:unk, 1130:endoftext)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [ ]:
print(tokenizer.decode(tokenizer.encode(text)))  # 디코딩

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


## 2.5 바이트 페어 인코딩